In [71]:
!pip install -q sentence-transformers faiss-cpu pandas
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [72]:

from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [73]:
# Load model
model = SentenceTransformer("all-mpnet-base-v2")

# Load recipes dataset
df = pd.read_csv("/kaggle/input/recipes/13k-recipes.csv")

# Check first few rows
df.head()


,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [74]:
# Combine title + ingredients for better search accuracy
recipe_texts = df["Title"].astype(str) + " " + df["Ingredients"].astype(str)

# Compute embeddings
recipe_embeddings = model.encode(recipe_texts.tolist(), convert_to_tensor=True)

print("✅ Embeddings computed!")


Batches:   0%|          | 0/422 [00:00<?, ?it/s]

✅ Embeddings computed!


In [75]:
def find_best_match(query):
    # Encode query
    query_embedding = model.encode([query], convert_to_tensor=True)
    
    # Compute cosine similarities
    scores = cosine_similarity(query_embedding.cpu(), recipe_embeddings.cpu())[0]
    
    # Get best index
    best_idx = scores.argmax()
    
    return {
        "Title": df.iloc[best_idx]["Title"],
        "Score": float(scores[best_idx]),
        "Ingredients": df.iloc[best_idx]["Ingredients"],
        "Instructions": df.iloc[best_idx]["Instructions"]
    }


In [76]:
while True:
    user_query = input("🔍 Enter your recipe search (type 'exit' to quit): ")
    if user_query.strip().lower() == "exit":
        print("Goodbye! 👋")
        break
    
    result = find_best_match(user_query)
    print(f"\n🍽 {result['Title']}\n   Score: {result['Score']:.4f}")
    print(f"   Ingredients: {result['Ingredients']}")
    print(f"   Instructions: {result['Instructions']}...\n")


🔍 Enter your recipe search (type 'exit' to quit):  potato salad


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🍽 Creamy Potato Salad with Lemon and Fresh Herbs
   Score: 0.6936
   Ingredients: ['3 pounds baby red potatoes', '3 tablespoons unseasoned rice vinegar', '3/4 cup mayonnaise', '3 medium green onions, thinly sliced', '1 celery stalk, cut into 1/3-inch cubes', '1/4 cup chopped fresh parsley', '1/4 cup chopped fresh basil', '2 tablespoons chopped fresh dill', '1 1/2 teaspoons finely grated lemon peel']
   Instructions: Bring potatoes to boil in large pot of water. Reduce heat to medium-low and simmer until potatoes are tender, about 17 minutes. Drain; let stand until cool enough to handle, about 20 minutes.
Cut potatoes into 3/4-inch pieces. Place 1 layer of potatoes in large bowl; sprinkle with some of vinegar and salt and pepper. Continue layering potatoes with vinegar, salt, and pepper. Add all remaining ingredients; toss. Season with salt and pepper. DO AHEAD Can be made 8 hours ahead. Cover and chill....



🔍 Enter your recipe search (type 'exit' to quit):  noodles


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🍽 Noodles with Chilled Tomato Broth
   Score: 0.5858
   Ingredients: ['1 1/2 lb. very ripe red tomatoes, chopped', '1 small garlic clove, smashed', '3 Tbsp. unseasoned rice vinegar', '1 Tbsp. sugar', '1 Tbsp. plus 1 tsp. kosher salt, plus more', '10 oz. thin noodles (preferably somen)', '5 large eggs, beaten to blend', '1/2 Tbsp. unsalted butter', '2 medium Persian cucumbers, halved, thinly sliced on a diagonal', '3 scallions, thinly sliced', '1/2 cup cilantro leaves with tender stems', 'Hot chili paste (such as sambal oelek; for serving)']
   Instructions: Blend tomatoes, garlic, vinegar, sugar, 1 Tbsp. salt, and 2 cups water in a blender on low speed until tomato pieces are no larger than 1/4" (it’s important to blend on low; otherwise, the tomato broth will foam up). Strain through a fine-mesh sieve into a large measuring cup, pressing on solids with a spoon to extract as much liquid as possible (you should have about 4 cups broth). Chill until ready to use.
Meanwhile, cook noodles

🔍 Enter your recipe search (type 'exit' to quit):  exit


Goodbye! 👋
